<a href="https://colab.research.google.com/github/piyuss/hyperspectral-image-demo/blob/main/hyper_image_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tutorial: Hyperspectral Image Analysis

In this tutorial, we will go through some common steps in hyperspectral image processing. First, let's download the files we will work with. Hyperspectral data tends to run the code cell below by clicking on the **Play icon** to the left of the cell. You can also run the cell by clicking on the cell and pressing **CTRL/CMD+ENTER**.

In [ ]:
!wget https://www.dropbox.com/sh/n6823u5i4mpc7dq/AAA5GmrwOR-AqsdyOxH47k60a?dl=1

In [10]:
!ls

'AAA5GmrwOR-AqsdyOxH47k60a?dl=0'   sample_data
